In [ ]:
import cv2
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource

# Define CLAHE functions
def rgb_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    channels = cv2.split(image)
    cl_channels = [clahe.apply(channel) for channel in channels]
    cl_image = cv2.merge(cl_channels)
    return cl_image

def lab_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe.apply(l_channel)
    merged = cv2.merge((cl, a_channel, b_channel))
    rgb_image = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
    return rgb_image

def max_green_gsc_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    green_channel = image[:, :, 1]
    max_pixel_image = np.maximum.reduce([image[:, :, 0], image[:, :, 1], image[:, :, 2]])
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    gray_clahe = clahe.apply(gray_image)
    green_clahe = clahe.apply(green_channel)
    max_pixel_clahe = clahe.apply(max_pixel_image)
    max_green_gsc_clahe_image = cv2.merge([gray_clahe, green_clahe, max_pixel_clahe])
    return max_green_gsc_clahe_image

# Load input image
file_in = '../data/image_4.jpg'
input_image = cv2.imread(file_in)

# Apply CLAHE methods
rgb_clahe_image = rgb_clahe(input_image)
lab_clahe_image = lab_clahe(input_image)
max_green_gsc_clahe_image = max_green_gsc_clahe(input_image)

# Convert images for Bokeh display
def cvt2bokeh(img):
    h, w = img.shape[:2]
    container = np.empty((h, w), dtype=np.uint32)
    view = container.view(dtype=np.uint8).reshape((h, w, 4))
    view[:, :, 0] = img[::-1, :, 0]  # copy red channel
    view[:, :, 1] = img[::-1, :, 1]  # copy blue channel
    view[:, :, 2] = img[::-1, :, 2]  # copy green channel
    view[:, :, 3] = 255
    return container

rgb_bokeh = cvt2bokeh(cv2.cvtColor(rgb_clahe_image, cv2.COLOR_BGR2RGB))
lab_bokeh = cvt2bokeh(cv2.cvtColor(lab_clahe_image, cv2.COLOR_BGR2RGB))
max_green_gsc_bokeh = cvt2bokeh(cv2.cvtColor(max_green_gsc_clahe_image, cv2.COLOR_BGR2RGB))

# Setup Bokeh output
output_notebook()

# Create ColumnDataSource for Bokeh
source1 = ColumnDataSource(data=dict(image=[rgb_bokeh]))
source2 = ColumnDataSource(data=dict(image=[lab_bokeh]))
source3 = ColumnDataSource(data=dict(image=[max_green_gsc_bokeh]))

# Calculate image dimensions
image_height, image_width = input_image.shape[:2]

# Create Bokeh figures
p1 = figure(title="RGB-CLAHE", 
            x_range=(0, image_width), y_range=(0, image_height), 
            width=image_width//6, height=image_height//6)
p1.image_rgba(image='image', source=source1, x=0, y=0, dw=image_width, dh=image_height)

p2 = figure(title="Lab-CLAHE", 
            x_range=p1.x_range, y_range=p1.y_range, 
            width=image_width//6, height=image_height//6)
p2.image_rgba(image='image', source=source2, x=0, y=0, dw=image_width, dh=image_height)

p3 = figure(title="MaxGreenGsc-CLAHE", 
            x_range=p1.x_range, y_range=p1.y_range,
            width=image_width//6, height=image_height//6)
p3.image_rgba(image='image', source=source3, x=0, y=0, dw=image_width, dh=image_height)

# Create grid layout
grid = gridplot([[p1, p2, p3]])

# Show grid layout
show(grid)
